In [2]:
import pandas as pd
import re
df=pd.read_parquet('../../data/transcriptions.parquet')
df['text']=df['text'].str.replace(r"\[.*?\]","",regex=True,flags=re.DOTALL).str.strip()


In [3]:
from nltk.util import ngrams
from collections import Counter, defaultdict
counts=defaultdict(Counter)
for text in df.text.dropna():
    tokens=text.lower().split()
    for w1,w2,w3 in ngrams(tokens,3):
        counts[(w1,w2)][w3]+=1  



In [4]:
import random as rd
starting_phrase='make America Great'
l=starting_phrase.lower().split()
L=l
a=counts[tuple(l[-2:])].most_common(1)[0][0]
L.append(a)
L
probas=[j/sum(j for i,j in counts[tuple(l[-2:])].most_common()) for i,j in counts[tuple(l[-2:])].most_common()]

fdr=[sum(probas[:i]) for i in range(len(probas))]
a=rd.random()
fdr
print(a)
print(fdr)
index=0
while a>fdr[index]:
    index+=1
index
counts[tuple(l[-2:])].most_common()[index][0]


0.6753337555636428
[0, 0.297148114075437, 0.3827046918123275, 0.4563017479300828, 0.5234590616375345, 0.5814167433302668, 0.6301747930082797, 0.6660533578656854, 0.6927322907083717, 0.7102115915363386, 0.7258509659613616, 0.7414903403863846, 0.7543698252069918, 0.7635694572217112, 0.7727690892364306, 0.781048758049678, 0.7893284268629255, 0.7957681692732291, 0.8022079116835327, 0.8086476540938363, 0.8150873965041399, 0.8206071757129715, 0.8261269549218032, 0.8307267709291629, 0.8353265869365226, 0.8399264029438823, 0.84360625574977, 0.8472861085556578, 0.8509659613615456, 0.8546458141674333, 0.8583256669733211, 0.8620055197792089, 0.8656853725850966, 0.8693652253909844, 0.8721251149954002, 0.874885004599816, 0.8776448942042319, 0.8804047838086477, 0.8831646734130635, 0.8859245630174793, 0.8886844526218951, 0.891444342226311, 0.8942042318307268, 0.8969641214351426, 0.8997240110395585, 0.9024839006439743, 0.9043238270469182, 0.906163753449862, 0.908003679852806, 0.9098436062557498, 0.911

'you'

In [5]:
def predict(starting_phrase,n):
    l=starting_phrase.lower().split()
    m=len(l)
    L=l
    a=0
    for i in range(n-m):
        L.append(counts[tuple(L[-2:])].most_common(1)[0][0])
    return " ".join(L)

predict("Make America Great",6)


'make america great again. thank you,'

In [6]:
predict("Women are",10)

'women are going to be a great job. and i'

# Adding Randomness

In [7]:
import random
def predict(starting_phrase,n):
    words=starting_phrase.lower().split()
    m=len(words)
    for i in range(n-m):
        context=tuple(words[-2:])
        probas=[j/sum(j for i,j in counts[context].most_common()) for i,j in counts[context].most_common()]
        fdr=[sum(probas[:i]) for i in range(len(probas))]
        a=rd.random()
        index=0
        while a>fdr[index]:
            index+=1
        words.append(counts[context].most_common()[index][0])
    return " ".join(words)

predict("Make America Great",6)


IndexError: list index out of range

# Constructing a sentence

In [20]:
texts = df.text.str.replace(r"([.!?])", r" \1 <EOS> ", regex=True)
tokens = texts.str.split()
tokens

from nltk.util import ngrams
from collections import Counter, defaultdict
counts=defaultdict(Counter)
for w1,w2,w3 in ngrams(tokens,3):
     counts[(w1,w2)][w3]+=1  


TypeError: unhashable type: 'list'